In [ ]:
'''
【项目09】  中国姓氏排行研究

作业要求
1、数据清洗、整合
要求：
① 将“data01”、“data02”分别读取，并且合并成一个数据
② 结合“户籍地城市编号”及“中国城市代码对照表”数据，将城市经纬度连接进数据中
③ 分别提取“工作地”中的省、市
提示：
① 可以先读取“data01”、“data02”，然后用pd.concat()来连接数据
② 新建字段“工作地-省”，“工作地-市”，“工作地-区县”,如果数据中“工作地”字段无法提取省和市，则用“未识别”填充单元格
   * 通过查看识别后的单元格，如果字数超过5则为“未识别”

2、查看姓氏“普遍指数”，普遍指数=姓氏人口数量
要求：
① 将数据按照“姓”做统计，找到数量最多的TOP20
② 分别制作图表，查看姓氏TOP20的数量及占比
   * 建议用bokeh出柱状图，并且为联动图表
③ 查看“王”姓的全国分布
   * 这里导出excel高版本文件，用powermap查看，绘制密度图
   * 同时可以尝试用echarts绘制空间柱状图来查看
④ 查看“姬”姓的全国分布
   * 这里导出excel高版本文件，用powermap查看，绘制密度图
   * 同时可以尝试用echarts绘制空间柱状图来查看
提示：
① bokeh中绘制联动图表时用gridplot
② powermap需要office2016的excel才会有，并且必须存储xlsx格式
③ powermap中需要通过在“值”中设置“姓的计数”才能正确显示热力图
④ powermap中可以通过“主题”来调节配色风格 / “平面地图”选项来调整球面可视化或者平面可视化
⑤ echarts绘制图表之前，需要对数据按照“lng”（或者“lat”）汇总，得到同一个地点的该姓氏人数，然后绘图
⑥ ecahrts通过设置以下参数来达到效果：视角远近、点柱大小

3、查看姓氏“奔波指数”，奔波指数=姓氏人均迁徙距离。迁徙距离为户籍地所在地级市与现居住地所在地级市的距离。
要求：
① 根据识别的工作地，通过Geocoding查询到对应坐标
② 选择一个姓氏，计算并查看其姓氏的奔波指数，并计算该姓氏的人均通勤距离
   * 在python中筛选数据之后，qgis内做空间分析
③ 按照起点和终点做计数，汇总同一个迁徙路径的数据
④ 通过echart制作通勤OD图
   * 可以将生成的line文件导出geojson，用空间线性轨迹图来表示
   * 这里线的valye为该迁徙路径的汇总计数   
提示：
① 可以筛选一些好玩的姓氏：汤、朴、廉、何、叶、冉等等 
② 需要对数据的工作地进行筛选，其中“工作地_市”、“工作地_区县”未识别的数据删除掉
③ 导出数据时，尽量columns名用全英文，避免qgis中出现乱码
④ 计算人均通勤距离的时候，需要删除掉户籍地与工作地相同的人（未迁移的人）
⑤ 在官网metrodata.cn的小工具中找到geocoding
⑥ qgis中需要安装插件“LinePlotter”来转线，并在qgis中计算平均通勤距离（需要投影，投影经度带可选48）
⑦ shapefile转geojson时：
   * 注意shapefile文件要投影回wgs84地理坐标系
   * 删除乱码子弹
   * 删除路径距离为0的数据


'''

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
% matplotlib inline

import warnings
warnings.filterwarnings('ignore') 
# 不发出警告

from bokeh.io import output_notebook
output_notebook()
# 导入notebook绘图模块

from bokeh.plotting import figure,show
from bokeh.models import ColumnDataSource
# 导入图表绘制、图标展示模块
# 导入ColumnDataSource模块

Loading BokehJS ...

In [ ]:
'''
1、数据清洗、整合
要求：
① 将“data01”、“data02”分别读取，并且合并成一个数据
② 结合“户籍地城市编号”及“中国城市代码对照表”数据，将城市经纬度连接进数据中
③ 分别提取“工作地”中的省、市
提示：
① 可以先读取“data01”、“data02”，然后用pd.concat()来连接数据
② 新建字段“工作地-省”，“工作地-市”，“工作地-区县”,如果数据中“工作地”字段无法提取省和市，则用“未识别”填充单元格
   * 通过查看识别后的单元格，如果字数超过5则为“未识别”

'''

In [2]:
# 查看数据,数据清洗

import os
os.chdir('C:/Users/Hjx/Desktop/项目09中国姓氏排行研究/')
# 创建工作路径

df01 = pd.read_csv('data01.csv',encoding = 'utf-8')
df02 = pd.read_csv('data02.csv',encoding = 'utf-8')
df_city = pd.read_excel('中国行政代码对照表.xlsx')
# 读取数据

df = pd.concat([df01,df02])
df = pd.merge(df,df_city,left_on='户籍地城市编号',right_on = '行政编码')  
df['工作地'] = df['工作地'].str[:15]   # 只提取工作地前15个字符即可
del df['行政编码']
del df['户籍地城市编号']
# 合并数据，添加经纬度字段
# 删除无用字段

print('读取数据共%i条' % len(df))
df.head(20)

读取数据共1006944条


,姓,工作地,省,市,区/县,lng,lat
0,王,沈阳市皇姑区塔湾街83-15号,辽宁省,沈阳,皇姑区,123.415376,41.848913
1,刘,虹口区四川北路1851号12,辽宁省,沈阳,皇姑区,123.415376,41.848913
2,李,沈阳市皇姑区香炉山路3-1号2,辽宁省,沈阳,皇姑区,123.415376,41.848913
3,王,沈阳市皇姑区崇山中路,辽宁省,沈阳,皇姑区,123.415376,41.848913
4,刘,沈阳市皇姑区梅江东街11号4-,辽宁省,沈阳,皇姑区,123.415376,41.848913
5,王,章武县章武镇,辽宁省,沈阳,皇姑区,123.415376,41.848913
6,张,沈阳市黄姑区金沙江街39号5-,辽宁省,沈阳,皇姑区,123.415376,41.848913
7,刘,铁西区腾飞一街31号,辽宁省,沈阳,皇姑区,123.415376,41.848913
8,阎,沈阳市皇姑区明廉路一72-4-,辽宁省,沈阳,皇姑区,123.415376,41.848913
9,雷,沈阳市皇姑区,辽宁省,沈阳,皇姑区,123.415376,41.848913


In [9]:
# 分别提取“工作地”中的省、市、区县
# 新建字段“工作地-省”，“工作地-市”，“工作地-区县”,如果数据中“工作地”字段无法提取省和市，则用“未识别”填充单元格
# *通过查看识别后的单元格，如果字数超过5则为“未识别”

df['工作地_省'] = df['工作地'].str.split('省').str[0]  
# 识别工作地-省

df['工作地_市'] = df['工作地'].str.split('省').str[1].str.split('市').str[0]             
df['工作地_市'][df['工作地_省'].str.len() > 5] = df['工作地_省'].str.split('市').str[0]  
# 识别工作地-市
# 在未识别出省的数据中，可能会有市的信息

df['工作地_区县'] = ''
df['工作地_区县'][(df['工作地_市'].str.len() < 5)&(df['工作地'].str.contains('区'))] = df['工作地'].str.split('市').str[1].str.split('区').str[0] + '区'  
df['工作地_区县'][(df['工作地_市'].str.len() > 5)&(df['工作地'].str.contains('区'))] = df['工作地'].str.split('区').str[0] + '区'  
df['工作地_区县'][(df['工作地_市'].str.len() < 5)&(df['工作地'].str.contains('县'))] = df['工作地'].str.split('市').str[1].str.split('县').str[0] + '县'  
df['工作地_区县'][(df['工作地_市'].str.len() > 5)&(df['工作地'].str.contains('县'))] = df['工作地'].str.split('县').str[0] + '县'
# 识别工作地-区县

df['工作地_省'][df['工作地_省'].str.len() > 5] = '未识别'
df['工作地_市'][df['工作地_市'].str.len() > 5] = '未识别'
df['工作地_区县'][(df['工作地_区县'].str.len() > 5) | (df['工作地_区县'].str.len() < 2)] = '未识别'
# 整理未识别单元格

df.columns = ['姓','工作地','户籍所在地_省','户籍所在地_市','户籍所在地_区县','户籍所在地_lng','户籍所在地_lat',
              '工作地_省','工作地_市','工作地_区县']
print('读取数据共%i条' % len(df))
# 数据整理
df.head(20)

读取数据共1006944条


,姓,工作地,户籍所在地_省,户籍所在地_市,户籍所在地_区县,户籍所在地_lng,户籍所在地_lat,工作地_省,工作地_市,工作地_区县
0,王,沈阳市皇姑区塔湾街83-15号,辽宁省,沈阳,皇姑区,123.415376,41.848913,未识别,沈阳,皇姑区
1,刘,虹口区四川北路1851号12,辽宁省,沈阳,皇姑区,123.415376,41.848913,未识别,未识别,虹口区
2,李,沈阳市皇姑区香炉山路3-1号2,辽宁省,沈阳,皇姑区,123.415376,41.848913,未识别,沈阳,皇姑区
3,王,沈阳市皇姑区崇山中路,辽宁省,沈阳,皇姑区,123.415376,41.848913,未识别,沈阳,皇姑区
4,刘,沈阳市皇姑区梅江东街11号4-,辽宁省,沈阳,皇姑区,123.415376,41.848913,未识别,沈阳,皇姑区
5,王,章武县章武镇,辽宁省,沈阳,皇姑区,123.415376,41.848913,未识别,未识别,章武县
6,张,沈阳市黄姑区金沙江街39号5-,辽宁省,沈阳,皇姑区,123.415376,41.848913,未识别,沈阳,黄姑区
7,刘,铁西区腾飞一街31号,辽宁省,沈阳,皇姑区,123.415376,41.848913,未识别,未识别,铁西区
8,阎,沈阳市皇姑区明廉路一72-4-,辽宁省,沈阳,皇姑区,123.415376,41.848913,未识别,沈阳,皇姑区
9,雷,沈阳市皇姑区,辽宁省,沈阳,皇姑区,123.415376,41.848913,未识别,沈阳,皇姑区


In [ ]:
'''
2、查看姓氏“普遍指数”，普遍指数=姓氏人口数量
要求：
① 将数据按照“姓”做统计，找到数量最多的TOP20
② 分别制作图表，查看姓氏TOP20的数量及占比
   * 建议用bokeh出柱状图，并且为联动图表
③ 查看“王”姓的全国分布
   * 这里导出excel高版本文件，用powermap查看，绘制密度图
   * 同时可以尝试用echarts绘制空间柱状图来查看
④ 查看“姬”姓的全国分布
   * 这里导出excel高版本文件，用powermap查看，绘制密度图
   * 同时可以尝试用echarts绘制空间柱状图来查看
提示：
① bokeh中绘制联动图表时用gridplot
② powermap需要office2016的excel才会有，并且必须存储xlsx格式
③ powermap中需要通过在“值”中设置“姓的计数”才能正确显示热力图
④ powermap中可以通过“主题”来调节配色风格 / “平面地图”选项来调整球面可视化或者平面可视化
⑤ echarts绘制图表之前，需要对数据按照“lng”（或者“lat”）汇总，得到同一个地点的该姓氏人数，然后绘图
⑥ ecahrts通过设置以下参数来达到效果：视角远近、点柱大小

'''

In [4]:
# 将数据按照“姓”做统计，找到数量最多的TOP10

name_count = df['姓'].value_counts()[:20]
result1_01 = pd.DataFrame({'count':name_count, 'count_pre':name_count/name_count.sum()})
# 筛选top20的姓氏，计数并计算占比
result1_01

,count,count_pre
王,79320,0.143500
张,72352,0.130894
李,66367,0.120067
刘,50310,0.091017
陈,45816,0.082887
杨,28385,0.051352
赵,21493,0.038884
周,20657,0.037371
吴,20376,0.036863
徐,19565,0.035396


In [5]:
# bokeh出联动柱状图

from bokeh.models import HoverTool
from bokeh.layouts import gridplot
# 导入模块

name_lst = result1_01.index.tolist()
source = ColumnDataSource(result1_01)
# 创建ColumnDataSource数据

hover1 = HoverTool(tooltips=[("姓氏计数", "@count")])  # 设置标签显示内容
result1 = figure(plot_width=800, plot_height=250,x_range = name_lst,
                 title="中国姓氏TOP20 - 计数" ,
                 tools=[hover1,'reset,xwheel_zoom,pan'])   # 构建绘图空间
result1.vbar(x='index', top='count', source=source,width=0.9, alpha = 0.7,color = 'red')   
result1.ygrid.grid_line_dash = [6, 4]
result1.xgrid.grid_line_dash = [6, 4]
# 柱状图1

hover2 = HoverTool(tooltips=[("姓氏占比", "@count_pre")])  # 设置标签显示内容
result2 = figure(plot_width=800, plot_height=250,x_range = result1.x_range,
                 title="中国姓氏TOP20 - 占比" ,
                 tools=[hover2,'reset,xwheel_zoom,pan']) 
result2.vbar(x='index', top='count_pre', source=source,width=0.9, alpha = 0.7,color = 'green')   
result2.ygrid.grid_line_dash = [6, 4]
result2.xgrid.grid_line_dash = [6, 4]
# 柱状图2

p = gridplot([[result1], [result2]])
# 组合图表
show(p)

In [8]:
# 查看“王”姓的全国分布

data_wang1 = df[df['姓'] == '王']
writer = pd.ExcelWriter('C:/Users/Hjx/Desktop/wang1.xlsx')
data_wang1.to_excel(writer,'sheet1',index=False)
writer.save()
# 导出数据1

data_wang2 = data_wang1.groupby(['姓','户籍所在地_lng','户籍所在地_lat'])['户籍所在地_市'].count()
data_wang2 = data_wang2.reset_index()
writer = pd.ExcelWriter('C:/Users/Hjx/Desktop/wang2.xlsx')
data_wang2.to_excel(writer,'sheet1',index=False)
writer.save()
# 导出数据2

print('导出完成！')

# 结论 → 老王们无处不在啊！

导出完成！


In [9]:
# 查看“姬”姓的全国分布

data_ji1 = df[df['姓'] == '姬']
writer = pd.ExcelWriter('C:/Users/Hjx/Desktop/ji1.xlsx')
data_ji1.to_excel(writer,'sheet1',index=False)
writer.save()
# 导出数据1

data_ji2 = data_ji1.groupby(['姓','户籍所在地_lng','户籍所在地_lat'])['户籍所在地_市'].count()
data_ji2 = data_ji2.reset_index()
writer = pd.ExcelWriter('C:/Users/Hjx/Desktop/ji2.xlsx')
data_ji2.to_excel(writer,'sheet1',index=False)
writer.save()
# 导出数据2

print('导出完成！')

# 结论 → “姬”传说是黄帝之姓、周朝国姓，并且是10大姓中7个姓的起源
# 千年过去，姬姓后嗣多已改为他姓，开枝散叶。而还保留着这个古老姓氏的人口，也仍然栖息在古老中华文明的发源地——河南。

导出完成！


In [14]:
'''
3、查看姓氏“奔波指数”，奔波指数=姓氏人均迁徙距离。迁徙距离为户籍地所在地级市与现居住地所在地级市的距离。
要求：
① 根据识别的工作地，通过Geocoding查询到对应坐标
② 选择一个姓氏，计算并查看其姓氏的奔波指数，并计算该姓氏的人均通勤距离
   * 在python中筛选数据之后，qgis内做空间分析
③ 按照起点和终点做计数，汇总同一个迁徙路径的数据
④ 通过echart制作通勤OD图
   * 可以将生成的line文件导出geojson，用空间线性轨迹图来表示
   * 这里线的valye为该迁徙路径的汇总计数   
提示：
① 可以筛选一些好玩的姓氏：汤、朴、廉、何、叶、冉等等 
② 需要对数据的工作地进行筛选，其中“工作地_市”、“工作地_区县”未识别的数据删除掉
③ 导出数据时，尽量columns名用全英文，避免qgis中出现乱码
④ 计算人均通勤距离的时候，需要删除掉户籍地与工作地相同的人（未迁移的人）
⑤ 在官网metrodata.cn的小工具中找到geocoding
⑥ qgis中需要安装插件“LinePlotter”来转线，并在qgis中计算平均通勤距离（需要投影，投影经度带可选48）
⑦ shapefile转geojson时，注意shapefile文件要投影回wgs84地理坐标系

'''

In [10]:
# 计算并查看“朴”姓氏的奔波指数

data_tang = df[['姓','户籍所在地_lng','户籍所在地_lat','工作地_市','工作地_区县']][df['姓'] == '汤']
data_tang = data_tang[data_tang['工作地_市'] != '未识别']
data_tang = data_tang[data_tang['工作地_区县'] != '未识别']
data_tang.columns = ['familyname','birth_lng','birth_lat','work_city','work_district']
# 筛选并清洗数据

writer = pd.ExcelWriter('C:/Users/Hjx/Desktop/tang.xlsx')
data_tang.to_excel(writer,'sheet1',index=False)
writer.save()
# 导出数据

print('数据条数为%i条' % len(data_tang))
data_tang.head(10)

数据条数为1664条


,familyname,birth_lng,birth_lat,work_city,work_district
744,汤,123.415376,41.848913,沈阳,和平区
981,汤,123.415376,41.848913,沈阳,皇姑区
1075,汤,123.415376,41.848913,沈阳,皇姑区
1227,汤,123.415376,41.848913,沈阳,皇姑区
2257,汤,123.415376,41.848913,沈阳,皇姑区
7590,汤,121.858666,29.378771,宁波,鄙州区
11000,汤,116.239678,40.033162,贵阳,南明区
11481,汤,116.239678,40.033162,北京,海淀区
12064,汤,116.239678,40.033162,北京,海淀区
13068,汤,116.239678,40.033162,北京,朝阳区
